In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import networkx as nx
import csv
import pandas
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D  
from matplotlib.pyplot import pause
from scipy import sparse
import pickle
import pylab
import sys
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

sys.setrecursionlimit(2000)
#sys.tracebacklimit = 0

# Set tolerance
tolerance = 300
totalN = 0
fix = 0

#MODIFIED
pickle_in = open("edgesS.pickle","rb")
edges = pickle.load(pickle_in)
pickle_in = open("nodeCoListS.pickle","rb")
nodeCoList = pickle.load(pickle_in)
pickle_in = open("treeedgeslistS.pickle","rb")
treeedgeslist = pickle.load(pickle_in)
pickle_in = open("treesS.pickle","rb")
trees = pickle.load(pickle_in)


def f(treeID):
    treenum = treeID
    G = []
    #G= nx.DiGraph()
    G = nx.MultiDiGraph()

    # Generate node and edge arrays, node coordinate arrays
    nodes       = np.asarray(nodeCoList)
    treeEdge    = np.asarray(treeedgeslist[treenum])
    dsInt       = treeEdge[:,0].astype(int)
    usInt       = treeEdge[:,1].astype(int)
    #Set for MALF , Disabled
    flowsFlt    = treeEdge[:,3]
    preFlow = flowsFlt
    TNFlt       = treeEdge[:,4]
    NO3NFlt     = treeEdge[:,5]
    NH4NFlt     = treeEdge[:,6]
    lArray      = treeEdge[:,7]
    sOrder =    treeEdge[:,8]

    #Fix NaN values
    for i in range(len(flowsFlt)):
       if np.isnan(flowsFlt[i]):
           flowsFlt[i] = 0


    cx1         = nodes[dsInt,0]  
    cy1         = nodes[dsInt,1]
    cx2         = nodes[usInt,0]
    cy2         = nodes[usInt,1]

    # Extract river variables

    # Add node positions to dictionary
    posDict = {}
    posDict.update(zip(dsInt, np.column_stack((cx1,cy1))))
    posDict.update(zip(usInt, np.column_stack((cx2,cy2))))

    # Add nodes and edges to graph
    G.add_nodes_from(dsInt)
    G.add_nodes_from(usInt)
    G.add_edges_from(np.column_stack((dsInt,usInt)))

    badNodes = [17333,20180,20256,22848,23247,23813,24625,27326,27750,27191,27190,25980,25979,25811,26457,26460,26459,25831,25558,17486,25496,25540,26129,26639,29638,32594,33756,34623,35265,27123,27173,27226,25896,27507,27809,27785,27834,24899,24764,25005,25083]

    # add new attribute to all edges
    counter = 0
    edge_labels = dict()
    order_labels = dict()
    for src, dest in G.edges():
        G[src][dest][0]['TN']   = TNFlt[counter]
        G[src][dest][0]['NO3N'] = NO3NFlt[counter]
        G[src][dest][0]['NH4N'] = NH4NFlt[counter]
        G[src][dest][0]['FR']   = flowsFlt[counter]
        # Disabled
    #    if (src in badNodes):
    #        #Was 0.021
    #        G[src][dest][0]['FR'] = 0.010
    #        print(G[src][dest][0]['FR'])
        G[src][dest][0]['FR2'] = 0
        G[src][dest][0]['B'] = 0
        G[src][dest][0]['Edge'] = counter
        G[src][dest][0]['L'] = lArray[counter]
        edge_labels[(src, dest)] = counter
        order_labels[(src, dest)] = sOrder[counter]
        counter += 1

    #print(G.edges(data=True))
    #G.edges(data='TN')
    edges = G.edges()
    #weights = [G[u][v][0]['FR'] for u,v in edges]
    weights = [G[u][v][0]['FR'] for u,v in edges]
    colourN = [G[u][v][0]['TN'] for u,v in edges]

    # Draw graph
    #nx.draw(G,posDict,node_size=0.0,width =[np.log(1.1+x) for x in weights],edge_color = colourN,edge_cmap=plt.cm.rainbow,arrows=False)    
    # Check if tree
    nx.is_tree(G)

    ##Fix problem edge for Waikato
    #G.remove_edge(17333, 17330)
    #nx.is_tree(G)

    #--Recursive Flow Interpolator--

    #Set complete array status to false for all nodes
    complete = np.zeros(len(nodes))

    # Find first order one river segment for specified tree
    node = dsInt[0]
    while (G.in_edges(node) != []):
        node = G.in_edges(node)[0][0]

    # Find final output node for tree
    outNode = dsInt[0]
    while (G.out_edges(outNode) != []):
        #print(outNode)
        outNode = G.out_edges(outNode)[0][1]



    fix = 0
    totalN = 0

    # Recursive Function
    def recurse(tolerance, nodeID):
        global fix
        global totalN
        status = 1

        # Get downstream node
        ndown = G.out_edges(nodeID)[0][1]
        #print(ndown)
        # Scan number of node inputs
        ninputs = len(G.in_edges(ndown))

        # Verify interpolate status for upstream nodes
        for i in range(ninputs):
            #print(i)
            if (G.in_edges(G.in_edges(ndown)[i][0]) == [] or complete[G.in_edges(ndown)[i][0]] == 1):
                status = 1;
            else: 
                status = 0;
                break

        #Store interpolated value if condition met
        if status == 1:
            upflow = 0;
            #print(G.out_edges(ndown))

            #Stop if at end of tree
            if G.out_edges(ndown) == []:
                return  

            for i in range(ninputs):
                #check for NaN value and add flow if not NaN
                if ~np.isnan(G[G.edges(G.in_edges(ndown)[i][0])[0][0]][G.edges(G.in_edges(ndown)[i][0])[0][1]][0]['FR']):
                    upflow += G[G.edges(G.in_edges(ndown)[i][0])[0][0]][G.edges(G.in_edges(ndown)[i][0])[0][1]][0]['FR']             

                #Scan upstream until better data is found
                else:
                    nscan = G.in_edges(ndown)[0][0]
                    #If no upstream data, set flow to zero
                    if (G.in_edges(nscan) == []) and (np.isnan(G[G.edges(nscan)[0][0]][G.edges(nscan)[0][1]][0]['FR'])):
                        upflow += 0
                    #Otherwise add upstream flow data
                    else:
                        while np.isnan(G[G.edges(nscan)[0][0]][G.edges(nscan)[0][1]][0]['FR']):
                            nscan = G.in_edges(nscan)[0][0]
                        upflow += G[G.edges(nscan)[0][0]][G.edges(nscan)[0][1]][0]['FR']    
                complete[G.in_edges(ndown)[i][0]] = 1;


            #add flow if beyond threshold    
            if (np.abs(1-(upflow/G[G.edges(G.out_edges(ndown)[0][0])[0][0]][G.edges(G.out_edges(ndown)[0][0])[0][1]][0]['FR'])) > tolerance/100):
                G[G.edges(G.out_edges(ndown)[0][0])[0][0]][G.edges(G.out_edges(ndown)[0][0])[0][1]][0]['FR'] = upflow #Add to original flow dictionary
                G[G.edges(G.out_edges(ndown)[0][0])[0][0]][G.edges(G.out_edges(ndown)[0][0])[0][1]][0]['FR2'] = upflow #Modified dictionary
                fix = fix+1
            complete[ndown] = 1  
            totalN = totalN+1
            recurse(tolerance,ndown)

        #Otherwise go further up tree   
        else:
            #Call recursion on all unsolved branches
            for i in range(ninputs):
                if (complete[G.in_edges(ndown)[i][0]] != 1):
                    n2 = len(G.in_edges(G.in_edges(ndown)[i][0]))           
                    # Too computationally intense
                    for j in range(n2):
                        #print("Exploring", G.in_edges(G.in_edges(ndown)[i][0])[j][0])
                        nodeID = G.in_edges(G.in_edges(ndown)[i][0])[j][0]
                        recurse(tolerance,nodeID);


    #Call recursion
    #recurse(tolerance, node)

    recurse(tolerance, node)

    weights = [G[u][v][0]['FR'] for u,v in edges]
    wFlt = np.float64(weights)
    colourN = [G[u][v][0]['TN'] for u,v in edges]
    # Draw graph
    fig = plt.figure()  
    nx.draw(G,posDict,node_size=0.0,width =[np.log(3.5+x) for x in weights],edge_color = colourN,edge_cmap=plt.cm.rainbow,arrows=False)# with_labels=True, font_size=6)    
    plt.gca().set_aspect('equal', adjustable='box')
    fig.set_facecolor("#00000F")
    #nx.draw_networkx_edge_labels(G, posDict, edge_labels = edge_labels, font_size=10)
    #nx.draw_networkx_edge_labels(G, posDict, edge_labels = {(17333, 17330):2}, font_size=6)
    # Check if tree
    nx.is_tree(G)

    incidence_matrix = nx.incidence_matrix(G, oriented=True) 

    lengths = lArray
    flowsFlt = wFlt

    global nodeArc
    nodeArc = incidence_matrix.todense()
    
interact(f, treeID =([178,8,35,802,663]));


interactive(children=(Dropdown(description='treeID', options=(178, 8, 35, 802, 663), value=178), Output()), _d…